In [336]:
from importlib import reload
from nesi_unbalanced import unbalanced2
from libdatasets import *
import libdatasets
from libactive import active_split
from dotenv import load_dotenv; load_dotenv()
import numpy as np
import nesi_unbalanced
import scipy

In [14]:
reload(libdatasets); from libdatasets import *

In [302]:
X, y = anuran(None)

In [303]:
for name, count in zip(*np.unique(y, return_counts=True)):
    print(name.ljust(25), f"{count/y.shape[0]:.0%}")

AdenomeraAndre            9%
AdenomeraHylaedactylus    48%
Ameeregatrivittata        8%
HylaMinuta                4%
HypsiboasCinerascens      7%
HypsiboasCordobae         16%
LeptodactylusFuscus       4%
OsteocephalusOophagus     2%
Rhinellagranulosa         1%
ScinaxRuber               2%


In [304]:
y[y=='AdenomeraHylaedactylus'].shape[0]

3478

In [305]:
reload(nesi_unbalanced); from nesi_unbalanced import unbalanced2
X_labelled, X_unlabelled, Y_labelled, Y_oracle, X_test, Y_test =active_split(
    X, y, test_size=0.5, labeled_size=10, shuffle=True, mutator=partial(unbalanced2, amount=0.8), random_state=np.random
)

In [306]:
X = np.concatenate((X_labelled, X_unlabelled))
y = np.concatenate((Y_labelled, Y_oracle))

In [307]:
print("Train distribution")
for name, count in zip(*np.unique(y, return_counts=True)):
    print(name.ljust(25), f"{count/y.shape[0]:.0%}")

Train distribution
AdenomeraAndre            4%
AdenomeraHylaedactylus    80%
Ameeregatrivittata        3%
HylaMinuta                2%
HypsiboasCinerascens      2%
HypsiboasCordobae         6%
LeptodactylusFuscus       2%
OsteocephalusOophagus     1%
Rhinellagranulosa         0%
ScinaxRuber               1%


In [308]:
print("Test distribution")
for name, count in zip(*np.unique(Y_test, return_counts=True)):
    print(name.ljust(25), f"{count/Y_test.shape[0]:.0%}")

Test distribution
AdenomeraAndre            9%
AdenomeraHylaedactylus    48%
Ameeregatrivittata        7%
HylaMinuta                4%
HypsiboasCinerascens      7%
HypsiboasCordobae         15%
LeptodactylusFuscus       3%
OsteocephalusOophagus     2%
Rhinellagranulosa         1%
ScinaxRuber               2%


In [309]:
y.shape[0], Y_test.shape[0]

(2165, 3598)

# Dataset verification

In [345]:
reload(nesi_unbalanced); from nesi_unbalanced import matrix, unbalanced2

In [346]:
for name, dataset in matrix['datasets']:
    try:
        X_labelled, X_unlabelled, Y_labelled, Y_oracle, X_test, Y_test = active_split(
            *dataset(), mutator=partial(unbalanced2, amount=0.5), test_size=0.5, labeled_size=10, shuffle=True, random_state=np.random
        )
    except ValueError as e:
        print(name, "could not be split:", e)
        continue
    if isinstance(X_labelled, scipy.sparse.csr_matrix):
        X = scipy.sparse.vstack((X_labelled, X_unlabelled))
    else:
        X = np.concatenate((X_labelled, X_unlabelled))
    y = np.concatenate((Y_labelled, Y_oracle))
    print(name, "has a split", y.shape[0], Y_test.shape[0])
    
    print("Train distribution")
    for name, count in zip(*np.unique(y, return_counts=True)):
        print(str(name).ljust(25), f"{count/y.shape[0]:.0%}")
    
    print("Test distribution")
    for name, count in zip(*np.unique(Y_test, return_counts=True)):
        print(str(name).ljust(25), f"{count/Y_test.shape[0]:.0%}")

avila has a split 8688 10434
Train distribution
A                         50%
B                         0%
C                         1%
D                         3%
E                         9%
F                         16%
G                         4%
H                         4%
I                         7%
W                         0%
X                         4%
Y                         2%
Test distribution
A                         40%
B                         0%
C                         1%
D                         3%
E                         10%
F                         19%
G                         4%
H                         5%
I                         8%
W                         0%
X                         5%
Y                         3%
sensorless has a split 5499 29255
Train distribution
1.0                       5%
2.0                       5%
3.0                       5%
4.0                       5%
5.0                       5%
6.0                       5%
7.0   

C:\Users\Zac\Programming\python\research\libactive.py:70: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if klass not in Y_labelled:


anuran, sensorless, avila